In [3]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [4]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, FloatType
import time
import datetime
from pyspark.sql.functions import year

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [5]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [6]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [7]:
#Updating column types

df = df.withColumn("date_built", df["date_built"].cast(IntegerType()))
df = df.withColumn("price", df["price"].cast(FloatType()))
df = df.withColumn("bedrooms", df["bedrooms"].cast(IntegerType()))
df = df.withColumn("bathrooms", df["bathrooms"].cast(IntegerType()))
df = df.withColumn("sqft_living", df["sqft_living"].cast(IntegerType()))
df = df.withColumn("sqft_lot", df["sqft_lot"].cast(IntegerType()))
df = df.withColumn("floors", df["floors"].cast(IntegerType()))
df = df.withColumn("waterfront", df["waterfront"].cast(IntegerType()))
df = df.withColumn("view", df["view"].cast(IntegerType()))
df = df.withColumn("year_sold", year(df['date']))

df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: float (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)
 |-- year_sold: integer (nullable = true)



In [8]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')

df.show()

+--------------------+----------+----------+--------+--------+---------+-----------+--------+------+----------+----+---------+
|                  id|      date|date_built|   price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year_sold|
+--------------------+----------+----------+--------+--------+---------+-----------+--------+------+----------+----+---------+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923.0|       4|        3|       3167|   11733|     2|         1|  76|     2022|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628.0|       2|        2|       2235|   14384|     1|         0|  23|     2021|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866.0|       2|        2|       2127|   10575|     2|         0|   0|     2019|
|b672c137-b88c-48b...|2019-10-16|      2016|239895.0|       2|        2|       1631|   11149|     2|         0|   0|     2019|
|e0726d4d-d595-407...|2022-01-08|      2017|424418.0|       3|        2|       2249|   13878|     2|         0|

In [9]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("""
    SELECT year_sold as Sale_Year, cast(AVG(price) as numeric (9,2)) as Average_Price
    FROM home_sales
    WHERE bedrooms==4
    GROUP by year_sold
    ORDER by year_sold
    """
).show()

+---------+-------------+
|Sale_Year|Average_Price|
+---------+-------------+
|     2019|    300263.70|
|     2020|    298353.78|
|     2021|    301819.44|
|     2022|    296363.88|
+---------+-------------+



In [10]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""
    SELECT date_built as Year_Built, cast(AVG(price) as numeric (9,2)) as Average_Price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP by date_built
    ORDER by date_built
    """
).show()

+----------+-------------+
|Year_Built|Average_Price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|    288770.30|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [11]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql(
    """
    SELECT date_built as Year_Built, cast(AVG(price) as numeric (9,2)) as Average_Price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 and sqft_living >= 2000
    GROUP BY date_built
    ORDER BY date_built
    """
 ).show()


+----------+-------------+
|Year_Built|Average_Price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|    293965.10|
|      2017|    280317.58|
+----------+-------------+



In [12]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql(
    """
    SELECT view as View_Rating, cast(AVG(price) as numeric (9,2)) as Average_Price
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view desc
    """
 ).show()

print("--- %s seconds ---" % (time.time() - start_time))

+-----------+-------------+
|View_Rating|Average_Price|
+-----------+-------------+
|        100|   1026669.50|
|         99|   1061201.42|
|         98|   1053739.33|
|         97|   1129040.15|
|         96|   1017815.92|
|         95|   1054325.60|
|         94|   1033536.20|
|         93|   1026006.06|
|         92|    970402.55|
|         91|   1137372.73|
|         90|   1062654.16|
|         89|   1107839.15|
|         88|   1031719.35|
|         87|   1072285.20|
|         86|   1070444.25|
|         85|   1056336.74|
|         84|   1117233.13|
|         83|   1033965.93|
|         82|   1063498.00|
|         81|   1053472.79|
+-----------+-------------+
only showing top 20 rows

--- 2.1371958255767822 seconds ---


In [13]:
# 7. Cache the the temporary table home_sales.
spark.sql('cache table home_sales')

DataFrame[]

In [14]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [15]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql(
    """
    SELECT view as View_Rating, cast(AVG(price) as numeric (9,2)) as Average_Price
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view
    """
 ).show()

print("--- %s seconds ---" % (time.time() - start_time))


+-----------+-------------+
|View_Rating|Average_Price|
+-----------+-------------+
|          0|    403848.51|
|          1|    401044.25|
|          2|    397389.25|
|          3|    398867.60|
|          4|    399631.89|
|          5|    401471.82|
|          6|    395655.38|
|          7|    403005.77|
|          8|    398592.71|
|          9|    401393.34|
|         10|    401868.43|
|         11|    399548.12|
|         12|    401501.32|
|         13|    398917.98|
|         14|    398570.03|
|         15|    404673.30|
|         16|    399586.53|
|         17|    398474.49|
|         18|    399332.91|
|         19|    398953.17|
+-----------+-------------+
only showing top 20 rows

--- 0.8900518417358398 seconds ---


In [16]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy('date_built').parquet('home_sales',mode='overwrite')

In [17]:
# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet('home_sales')

In [18]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView('parquet_temp_home')

In [22]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql(
    """
    SELECT view as View_Rating, cast(AVG(price) as numeric (9,2)) as Average_Price
    FROM parquet_temp_home
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view desc
    """
 ).show()

print("--- %s seconds ---" % (time.time() - start_time))

+-----------+-------------+
|View_Rating|Average_Price|
+-----------+-------------+
|        100|   1026669.50|
|         99|   1061201.42|
|         98|   1053739.33|
|         97|   1129040.15|
|         96|   1017815.92|
|         95|   1054325.60|
|         94|   1033536.20|
|         93|   1026006.06|
|         92|    970402.55|
|         91|   1137372.73|
|         90|   1062654.16|
|         89|   1107839.15|
|         88|   1031719.35|
|         87|   1072285.20|
|         86|   1070444.25|
|         85|   1056336.74|
|         84|   1117233.13|
|         83|   1033965.93|
|         82|   1063498.00|
|         81|   1053472.79|
+-----------+-------------+
only showing top 20 rows

--- 0.6268987655639648 seconds ---


In [20]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table home_sales')

DataFrame[]

In [21]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached('home_sales'):
  print('home_sales is cached')
else:
  print('not cached')


not cached
